# RTED-TDS Co-Sim

Authored by [Jinning Wang](https://scholar.google.com/citations?user=Wr7nQZAAAAAJ&hl=en&oi=ao), recently updated on 06/06/2022.

This notebook is used to implement the Co-Simulation of Real Time Economic Dispatch (RTED) and Time-Domain Simulation (TDS).
In the RTED, SFR is considered. In the TDS, AGC is implemented with an equation defined PI controller.

RTED is seperated into two DCOPF and ACOPF: 1) DCOPF with SFR is solved in gurobipy, which is packaged into the class ``rted`` in the file ``rted.py``. 2) ACOPF is solved in pandapower.

Class ``rted2`` has attributes ``mdl`` which is a gurobipy model.

Table of contents:
- Import case: import ADNES case
- Load synthetic: build load curve as a scalar
- Setup ``ssp`` and ``ssd``
- Prepare: define some functions
- Define param: define RTED loop parameters
- Loop: Co-Sim loop

List of major vars:

- ``ssa`` ANDES system
- ``ssp`` pandapower net
- ``ssd`` DCOPF instance
- ``sse`` EV aggregator

Things need to be done when switching to other cases:
- Assign generator cost data. The rows of gen_cost array should be the same with the length of ``ssp.gen``
  including those uncontrollable ones, so the DCOPF model can build correctly.
- Define generator controllability of ``ssp``, all generators are controllable by default
- Define power system data: SFR cost, ramp limit (5-min-based)
- Define generator type in ``ssd``, default as type I. For type I generator, generator limtis have impact on both generation and SFR capacity. For yype II generator, SFR capacity is determined by param ``pru_max`` and ``prd_max``

Environment requirements:
- ANDES (1.6.3+)
- pandapower (2.7.0 is tested to be function correctly)
- gurobipy
- pandas, matplotlib, scipy

## Import case

In [ ]:
%run -i 'cosim_import.py'

`caseH` is the start time [H] of the co-simulation, now 10 and 18 are supported.

In [ ]:
caseH = 18
ict_off = True

%run -i 'cosim_setup_andes.py'

## Load synthetic

Create load data ``d_syn``, which is a ``DataFrame`` that have three columns: ``time``, ``s10``, ``h10``. ``time`` is by seconds, ``s10`` is scalar load.

In [ ]:
%run -i 'cosim_loadsyn.py'

## Setup ``ssp`` and ``ssd``

Convert ``ssa`` to pandapower net ``ssp``, add generator cost

The input cost array follow the matpower/pypower format, now only poly_cost is supported

In [ ]:
%run -i 'cosim_setup_dispatch.py'
dc_comp.round(4)

## Prepare

### Make link table

In [ ]:
%run -i 'cosim_linktable.py'
ssa_key2

## Loop

In the loop, there are mainly X parts:

- interval RTED: run DCOPF (``ssd.mdl``), run ACOPF(``ssp``), 

- interval AGC: do AGC, do dispatch with smooth setpoints

- interval PQ: alter load, run TDS(``ssa.TDS``)

Notes:

- The setpoints for DG are coded but not verified yet.

- After the development of ANDES control room, the dispatch and AGC part can be refactored.

- interface variables: ``DG.pmx``: DPV profile, ``DG.pref0``: setpoints, ``DG.pext0``: AGC

- ACOPF in pandapower considered generator limtis of ramping and SFR reserve

In [ ]:
# Change ``rru``, ``rrd``, ``rsfr``, ``t_total`` for necessary.
t_total = 400

rru, rrd, rsfr = 1, 0.0, 0.04

# Define functions and constants used in loop
%run -i 'cosim_const.py'

Check convergence

In [ ]:
%run -i 'cosim_bmain.py'

Loop

In [ ]:
%run -i 'cosim_main.py'

In [ ]:
ssa.exit_code

In [ ]:
sse.plot_agc()

In [ ]:
sse.plot()

In [10]:
sse.n_pref

1

In [ ]:
sse.plot_agc()

In [ ]:
plt.plot(range(49), agc_out.iloc[10].values[2:])

In [ ]:
agc_table.round(4)

In [ ]:
# ssa.TDS.save_output()

In [ ]:
%run -i 'cosim_plot.py'

In [ ]:
cosim_out.to_csv('cosim_18_H_out.csv', index=False)
sse_out.to_csv('cosim_18_H_sse.csv', index=False)
bu_df.to_csv('cosim_18_H_bu.csv', index=False)
bd_df.to_csv('cosim_18_H_bd.csv', index=False)
pg_df.to_csv('cosim_18_H_pg.csv', index=False)
agc_mile.to_csv('cosim_18_H_agcm.csv', index=False)
sfr_res.to_csv('cosim_18_H_sfr.csv', index=False)
ev_agc.to_csv('cosim_18_H_evagc.csv', index=False)
ev_soc.to_csv('cosim_18_H_evsoc.csv', index=False)
sse.ev.to_csv('cosim_18_H_evdata.csv', index=False)

import csv
new_path = open('cosim_18_H_rted.csv', 'w')
z = csv.writer(new_path)
for new_k, new_v in rted_res.items():
    z.writerow([new_k, new_v])
new_path.close()

In [ ]:
plt.style.use('ieee')
%run -i 'cosim_plotev.py'